# fpgaConvNet Optimiser Tutorial

The [fpgaConvNet Optimiser](https://github.com/AlexMontgomerie/fpgaconvnet-optimiser) is the frontend for generating the dataflow hardware configuration for a given convolutional neural network. In this tutorial, The main aspects to this optimiser will be covered. A step by step guide of stages to the model to hardware description is given. Instructions on using the command-line interface are given towards the end.

## 1. Initialising a Hardware Model

The first step is parsing the CNN model into an initial hardware model. This initial hardware model is a very basic implementation, which maps the CNN layers to their hardware equivalent. 

The first step is to intialise a `Network` model. This class the top-level in the hierarchy of hardware model abstractions.

In [ ]:
# setup environment
!python -m pip install fpgaconvnet-optimiser-AlexMontgomerie
!wget https://www.dropbox.com/s/9ua1diq2fd254ab/lenet.onnx
!mkdir figures
from IPython.display import Image

     |████████████████████████████████| 92kB 5.4MB/s 
     |███████████████████             | 462.9MB 1.2MB/s eta 0:04:12

In [ ]:
from fpgaconvnet_optimiser.models.network import Network

# create network
lenet = Network("lenet", "lenet.onnx")

# visualise network
lenet.visualise("figures/lenet_baseline.png")
Image(filename="figures/lenet_baseline.png") 

In [ ]:
# get all avaiable partitions
print("all horizontal splits: ", lenet.get_all_horizontal_splits(0))

# split all partitions
lenet.split_horizontal(0,("pool2","ip1_6relu1"))

# visualise network
lenet.visualise("figures/lenet_partitioned.png")
Image(filename="figures/lenet_partitioned.png") 

In [ ]:
# show rates before
print(f"rate in conv1 (fine = )")